In [2]:
%matplotlib inline

import Bio.PDB
import Bio.SeqIO
from Bio import pairwise2
from Bio.SubsMat import MatrixInfo as matlist
from Bio.SeqUtils import seq3

import re
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

# paths

dbfiles = glob.glob("../data/mhc_alleles/hla/*prot.fasta") + glob.glob("../data/mhc_alleles/mhc/*/*prot.fasta")
hla_fasta = '../data/mhc_alleles/hla/hla_prot.fasta'
mhc_db_path = '../sandbox/db.fasta'
epitope_db_path = '../data/epitopes/epitope_table.csv'
blast_output_path = '../sandbox/blast_output'
epitope_annot = '../sandbox/epitope_annot'
mhc_annot = '../sandbox/mhc_annot'
query_path = '../sandbox/query.fasta'
pdb_path = '../data/structures'
epitope_query_path = '../sandbox/epitope_query.fasta'
badlist = '../data/structures/badlist'

In [3]:
import __main__
__main__.pymol_argv = ['pymol','-Qc'] # Pymol: quiet and no GUI
from time import sleep
import pymol
from pymol import cmd

pymol.finish_launching()

In [4]:
# replace ' ' in indentifiers with ';'
db = open(mhc_db_path, 'w')
for dbfile in dbfiles:
    f = open(dbfile, 'r')
    text = f.read()
    f.close()
    db.write(';'.join(text.split(' ')))

db.close()

In [5]:
# fetch all chains from pdbs, combine in one query file and blast against hla db

def blast_pdb():
    parser = Bio.PDB.PDBParser()
    ppb = Bio.PDB.CaPPBuilder()

    query_file = open(query_path, 'w')

    pdbfiles = glob.glob(pdb_path + "/*.pdb")
    for pdb in pdbfiles:
        if pdb in pd.read_csv(badlist, sep='\t', header=None)[0]:
            continue
            
        structure = parser.get_structure(id=pdb[-8:-4], file=pdb)
        for chain in structure[0]:
            try:
                query_file.write('>%s;%s\n%s\n' % (structure.id, chain.id, ''.join([str(s.get_sequence()) for s in ppb.build_peptides(chain)])))
            except Exception:
                continue

    query_file.close()
    
    # submit query file for blast search
    command = ["blastp", "-query", query_path, 
                     "-subject", mhc_db_path, "-out", 
                     blast_output_path, "-outfmt", str(6), "-num_alignments", str(1)]
    if subprocess.call(command) != 0:
        print('Failed to blast')
        exit()
    
#blast_pdb()

In [6]:
# convert blast output to mhc table

def host_n_allele(s):
    host = s
    if ';' in s:
            host = s.split(';')[1]
    if '-' in host:
        return host.split('-')
    else:
        return ['Human', host]

def make_mhc_table():
    blasted = pd.read_csv(blast_output_path, sep='\t', header=None)
    pdb_list = list(set([x.split(';')[0] for x in blasted[0]]))
    
    mhc_table = []

    for pdb in pdb_list:
        piece = blasted[blasted[0].str.startswith(pdb)]
        if piece.shape[0] < 2:
            continue
        thres = np.mean(piece[11])
        chunk = piece[piece[11] > thres]
        for i in chunk.index:
            line = chunk.loc[i, :]
            if line[2] < 90:
                continue
            hna = host_n_allele(line[1])
            susp = 0
            if line[11] < thres or line[2] < 95:
                susp = 1
            row = [line[0].split(';')[0], line[0].split(';')[1], 'class', hna[0], hna[1], susp]
            mhc_table.append(row)

    mhc_table = pd.DataFrame(mhc_table, columns=['pdb', 'chain', 'mhc_class', 'mhc_host', 'mhc_allele', 'unsure'])
    mhc_table.to_csv(mhc_annot, sep='\t', index=False)
    
#make_mhc_table()

In [7]:
# make epitope table using epitope database

def make_epitope_table():
    epitope_db = pd.read_csv(epitope_db_path, header=None, skiprows=1, low_memory=False)
    epitope_table = []

    query_file = open(query_path, 'r')
    text = query_file.read()
    query_file.close()
    text = text.split()

    epitope_query_file = open(epitope_query_path, 'w')
    for i in range(len(text)):
        if text[i][0] == '>':
            if len(text[i+1]) < 30:
                seq = text[i+1]
                name = text[i].split(';')
                epitope_query_file.write('%s\n%s\n' % (text[i], seq))
                epitope_db_tmp = epitope_db[epitope_db[2].apply(lambda x: len(x)) == len(seq)]
                epitope_db_tmp = epitope_db_tmp[epitope_db_tmp[2].str.contains(seq)]
                trial = epitope_db_tmp[epitope_db_tmp[2] == seq]
                if trial.shape[0] > 0:
                    epitope_db_tmp = trial
                if epitope_db_tmp.shape[0] > 0:
                    row = [name[0][1:], name[1], seq, len(seq), epitope_db_tmp.iloc[0, 7], 
                           epitope_db_tmp.iloc[0, 9], epitope_db_tmp.iloc[0, 11]]
                else:
                    row = [name[0][1:], name[1], seq, len(seq), 'NaN', 'NaN', 'NaN']
                epitope_table.append(row)

    epitope_query_file.close()

    epitope_table = pd.DataFrame(epitope_table, columns=['pdb', 'chain', 'sequence', 'length', 'name', 'organism', 'comments'])
    epitope_table.to_csv(epitope_annot, sep='\t', index=False)

#make_epitope_table()

In [8]:
epitope_table = pd.read_csv(epitope_annot, sep='\t')
mhc_table = pd.read_csv(mhc_annot, sep='\t')

mhc_table = mhc_table[mhc_table.mhc_host == 'Human']

In [9]:
# make detailed mhc table

# check if chains are less than 8 A separated
def chain_close(a, b):
    mindist = 100
    for a1 in a.get_atoms():
        if mindist < 8:
            return True
        for a2 in b.get_atoms():
            dist = a1-a2
            if dist < mindist:
                mindist = dist
    return False

# mean distance
def chain_dist(a, b):
    ave = 0
    num = 0
    for a1 in a.get_atoms():
        mindist = 100
        for a2 in b.get_atoms():
            dist = a1-a2
            if dist < mindist:
                mindist = dist
        ave += mindist
        num += 1
    return ave/num

# pair MHC chains based on allele names and interchain distances
def pair_chains(mhc, epi, alleles, pdb):
    model = parser.get_structure(id=pdb[-8:-4], file=pdb)[0]
    out = []
    
    for epi_ch in epi:
        firmindist = 100
        first_id = first_al = ''
        for mhc_ch, allele in zip(mhc, alleles):
            dist = chain_dist(model[epi_ch], model[mhc_ch])
            if dist < firmindist:
                firmindist = dist
                first_id = mhc_ch
                first_al = allele
        
        if len(allele.split('*')[0]) > 2:
            second_id = second_al = ''
            secmindist = 100
            for mhc_ch, allele in zip(mhc, alleles):
                if first_id == mhc_ch: 
                    continue
                dist = chain_dist(model[epi_ch], model[mhc_ch])
                if dist < secmindist:
                    secmindist = dist
                    second_id = mhc_ch
                    second_al = allele
            if (firmindist < 8 and secmindist < 8 and chain_close(model[first_id], model[second_id])):
                if (first_al.split('*')[0][2] == 'B' and second_al.split('*')[0][2] == 'A'):
                    out.append([[second_id, first_id], epi_ch])
                else:
                    out.append([[first_id, second_id], epi_ch])
        else:
            out.append([[first_id], epi_ch])
    return out

# verify no intersection
def correctness(pairs):
    for i in range(len(pairs)-1):
        for j in range(i+1, len(pairs)):
            for l in range(len(pairs[i][0])):
                for m in range(len(pairs[j][0])):
                    assert(pairs[i][0][l] != pairs[j][0][m])


def make_annotation_table():
    parser = Bio.PDB.PDBParser(QUIET=True)
    ppb = Bio.PDB.CaPPBuilder()                    
    pdb_set = list(set(mhc_table.pdb))
    annotation = []

    for pdb in pdb_set:
        mhc_chunk = mhc_table[mhc_table.pdb == pdb]
        epi_chunk = epitope_table[epitope_table.pdb == pdb]
        pairs = pair_chains(list(mhc_chunk.chain), list(epi_chunk.chain), list(mhc_chunk.mhc_allele), 
                            "%s/%s.pdb" % (pdb_path, pdb))
        correctness(pairs)

        for pair in pairs:
            line = ''
            if len(pair[0]) == 1:
                mhc_local = mhc_chunk[mhc_chunk.chain == pair[0][0]]
                assert(mhc_local.iloc[0, 4][1] == '*')
                #               pdb          class         host              1 chain  
                line = [mhc_local.iloc[0, 0], 'I', mhc_local.iloc[0, 3],  mhc_local.iloc[0, 1], 
                #         1 chain allele       2 chain   2 chain allele      
                        mhc_local.iloc[0, 4],   '',         'B2M']

            if len(pair[0]) == 2:
                mhc_a = mhc_chunk[mhc_chunk.chain == pair[0][0]]
                mhc_b = mhc_chunk[mhc_chunk.chain == pair[0][1]]

                assert(mhc_a.iloc[0, 4][:2] == mhc_b.iloc[0, 4][:2])
                assert(mhc_a.iloc[0, 4][2] == 'A')
                assert(mhc_b.iloc[0, 4][2] == 'B')
                line = [mhc_chunk.iloc[0, 0], 'II', mhc_chunk.iloc[0, 3], mhc_a.iloc[0, 1], 
                        mhc_a.iloc[0, 4], mhc_b.iloc[0, 1], mhc_b.iloc[0, 4]]

            line = line + list(epi_chunk[epi_chunk.chain == pair[1]].iloc[0, 1:])
        annotation.append(line)

        cols = ['pdb', 'mhc_class', 'host', 'a_chain_id', 'a_chain_allele', 'b_chain_id', 'b_chain_allele', 
            'antigen_id', 'antigen_seq', 'antigen_len', 'antigen_name', 'antigen_organism', 'comments']
        annotation = pd.DataFrame(annotation, columns=cols).drop_duplicates()
        annotation.to_csv('../sandbox/detailed_mhc', index=None, sep='\t')
        
#make_annotation_table()

annotation = pd.read_csv('../sandbox/detailed_mhc', sep='\t')

In [10]:
# define microglobuline chains
def b2m_align(s):
    b2m = 'MSRSVALAVLALLSLSGLEAIQRTPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKDEYACRVNHVTLSQPKIVKWDRDM'
    matrix = matlist.blosum90
    gap_open = -1.0
    gap_extend = -0.5
    alns = pairwise2.align.globalds(s, b2m, matrix, gap_open, gap_extend)
    return alns[0]

def b2m_define(table):
    chunk = table[table.mhc_class == 'I']
    parser = Bio.PDB.PDBParser(QUIET=True)
    ppb = Bio.PDB.CaPPBuilder() 
    
    for i in chunk.index:
        pdb = chunk.loc[i, 'pdb']
        print(pdb)
        structure = parser.get_structure(id=pdb, file=pdb_path+'/'+pdb+'.pdb')[0]
        mhc = chunk.loc[i, 'a_chain_id']
        pep = chunk.loc[i, 'antigen_id']
        mindist = 1000
        b2m = None;
        for chain in structure.get_chains():
            if chain.get_id() != mhc and chain.get_id() != pep:
                if len(list(structure.get_chains())) == 3:
                    b2m = chain
                else:
                    chain_seq = ''.join([str(x.get_sequence()) for x in ppb.build_peptides(chain)])
                    if b2m_align(chain_seq)[2] > 400:
                        if chain_close(chain, structure[mhc]):
                            dist = chain_dist(chain, structure[mhc])
                            if dist <= mindist:
                                mindist = dist
                                b2m = chain
                                
        if b2m == None:
            table.loc[i, 'b_chain_id'] = None
            print('bsm chain = %s' % 'None')
        else:
            table.loc[i, 'b_chain_id'] = b2m.get_id()
            print('bsm chain = %s' % b2m.get_id())

    annotation.to_csv('../sandbox/detailed_mhc', index=None, sep='\t')
        
#b2m_define(annotation)

annotation = pd.read_csv('../sandbox/detailed_mhc', sep='\t')

In [11]:
iclass = annotation[annotation.mhc_class == 'I']
iclass = iclass[iclass.a_chain_allele.str.contains('[ABC]\*', regex=True)]
iclass.a_chain_allele = [':'.join(x.split(':')[:2]) for x in iclass.a_chain_allele]
iclass = iclass.reset_index(drop=True)

In [12]:
tmp = iclass[iclass.a_chain_allele.str.startswith('A*02:01')]
with open('../sandbox/peptides_cluster', 'w') as file:
    for i in range(tmp.shape[0]):
        file.write('>%s\n%s\n' % (tmp.iloc[i, 0], tmp.iloc[i, 8]))
        

cluster1 = ['2x4r', '3gsv', '3gsq', '3mrd', '3mrb', '3mrc', '3gsx', '3gsw', '3gsu', '3gsr', '3mr9']
cluster2 = ['4l3c', '1s9w', '3kla', '3gjf', '1s9y', '1s9x']

In [13]:
def align_pdb(table):
    
    class SelectChains(Bio.PDB.Select):
        def __init__(self, chain_letters):
            self.chain_letters = chain_letters
        def accept_chain(self, chain):
            return (chain.get_id() in self.chain_letters)

    parser = Bio.PDB.PDBParser(QUIET=True)

    name = table.iloc[0, 0]; amhc = table.iloc[0, 3]; pept = table.iloc[0, 7]; b2m = table.iloc[0, 5]
    ref = parser.get_structure(id=name, file=pdb_path+'/'+name+'.pdb')[0]
    ref_atoms = []
    for atom in ref[amhc].get_atoms():
        if atom.get_name() == 'CA':
            ref_atoms.append(atom)

    io = Bio.PDB.PDBIO()
    io.set_structure(ref)
    io.save('../sandbox/aligned_pdb/%s_cut.pdb' % name, SelectChains([amhc, pept, b2m]))

    super_imposer = Bio.PDB.Superimposer()

    for i in range(1, table.shape[0]):
        name = table.iloc[i, 0]
        amhc = table.iloc[i, 3]
        b2m  = table.iloc[i, 5]
        pept = table.iloc[i, 7]
        sample = parser.get_structure(id=name, file=pdb_path+'/'+name+'.pdb')[0]
        sample_atoms = []
        for atom in sample[amhc].get_atoms():
            if atom.get_name() == 'CA':
                sample_atoms.append(atom)
                
        length = min(len(ref_atoms), len(sample_atoms))
        super_imposer.set_atoms(ref_atoms[:length], sample_atoms[:length])
        super_imposer.apply(sample.get_atoms())
        
        io.set_structure(sample)
        io.save('../sandbox/aligned_pdb/%s_cut.pdb' % name, SelectChains([amhc, pept, b2m]))
        
#align_pdb(iclass.loc[:50, :])

In [14]:
iclass[(iclass.antigen_len == 9) & (iclass.a_chain_allele.str.startswith('A'))].head()

,pdb,mhc_class,host,a_chain_id,a_chain_allele,b_chain_id,b_chain_allele,antigen_id,antigen_seq,antigen_len,antigen_name,antigen_organism,comments
0,3i6l,I,Human,D,A*24:02,E,B2M,F,QFKDNVILL,9,Nucleoprotein,SARS coronavirus Tor2,NaN
1,5swq,I,Human,A,A*02:01,B,B2M,C,CVNGSCFTV,9,neuraminidase,Influenza A virus,NaN
5,3mri,I,Human,A,A*02:01,B,B2M,P,CINMWCWTV,9,NaN,NaN,NaN
7,2uwe,I,Human,H,A*02:01,I,B2M,J,ALWGFFPVL,9,chromosome 15 open reading frame 24,Homo sapiens,NaN
8,3gsw,I,Human,A,A*02:01,B,B2M,P,NLVPMVAAV,9,NaN,NaN,NaN


In [15]:
# if residue in mhc is close to antigen then return 1.
# if further return 0.5
# if far return 0.01
def get_points(res, chain):
    mdist = 1000
    for a in res:
        for b in chain.get_atoms():
            dist = a-b
            if dist < mdist:
                mdist = dist
                if mdist < 5:
                    return 1.0
    if mdist < 10:
        return 0.5
    return 0.01

# return sorted list of best alleles
def score_alignments(scores, blasted, points, alleles, line_id, size):
    p1 = re.compile('\w+\s+([0-9]+)\s+(\w+)\s+([0-9]+)\\n')
    p2 = re.compile('Subject_([0-9]+)')
    p3 = re.compile('[A-Z]')

    res_start = start = end = counter = 0
    
    for line in blasted:
        if line[:7] == 'Query_1':
            m = p1.match(line)
            res_start = int(m.group(1))-1
            start = m.start(2)
            end = m.end(2)
            counter = 0
            
        if line[:8] == 'Subject_':
            if counter >= size:
                break
            allele_id = int(p2.match(line).group(1))-1
            chunk = line[start:end+1]
            if np.isinf(scores[line_id, allele_id]):
                scores[line_id, allele_id] = 0.0
            
            for m in p3.finditer(chunk):
                res_ind = res_start+m.start()
                scores[line_id, allele_id] += points[res_ind]
            counter += 1

    return scores

# blast crystal hla I seqs against hla data base
# and create allele-pdb compatibility table
def make_allele2pdb_table():
    p4 = re.compile('.+([ABC]\*[0-9]+:[0-9A-Z]+)')
    alleles = []
    file = open("../data/mhc_alleles/hla/hla1class.fasta", 'r')
    lines = file.readlines()
    file.close()
    for line in lines:
        if line[0] == '>':
            #alleles.append(p4.search(line).group(1))
            alleles.append(line.split(';')[1])

    parser = Bio.PDB.PDBParser(QUIET=True)
    ppb = Bio.PDB.CaPPBuilder()

    size = len(alleles)
    num_alignments = size

    scores = np.zeros((iclass.shape[0], size))
    scores.fill(np.inf)

    for i in range(iclass.shape[0]):
        name = iclass.iloc[i, 0]
        print(name)
        chain = iclass.iloc[i, 3]
        antichain = iclass.iloc[i, 7]
        struct = parser.get_structure(id=name, file=pdb_path+'/'+name+'.pdb')[0]
        seq = ''.join([str(x.get_sequence()) for x in ppb.build_peptides(struct[chain])])
        query = open('../sandbox/query.fasta', 'w')
        query.write('>%s\n%s\n' % (name, seq))
        query.close()

        command = ["blastp", "-query", "../sandbox/query.fasta",
                   "-subject", "../data/mhc_alleles/hla/hla1class.fasta", "-out", 
                   "../sandbox/out", "-outfmt", str(3), "-num_alignments", str(num_alignments),
                   "-line_length", str(1000), "-ungapped", "-comp_based_stats", "F"]
        if subprocess.call(command) != 0:
            print('Failed to blast')
            exit()

        points = np.array([0.0]*len(struct[chain]))
        residues = list(struct[chain])
        for j in range(len(residues)):
            points[j] = get_points(residues[j], struct[antichain])

        file = open("../sandbox/out", 'r')
        blasted = file.readlines()
        file.close()

        scores = score_alignments(scores, blasted, points, alleles, i, num_alignments)

    alleles_vs_pdb = pd.DataFrame([alleles]+[list(x) for x in list(scores)]).T.drop_duplicates(0)
    alleles_vs_pdb.index = alleles_vs_pdb[0]
    alleles_vs_pdb = alleles_vs_pdb.iloc[:, 1:]
    alleles_vs_pdb.columns = list(iclass.pdb)
    alleles_vs_pdb = alleles_vs_pdb.replace(np.inf, np.nan)
    alleles_vs_pdb.to_csv('../sandbox/alleles_vs_pdb', sep='\t', float_format='%.2f')
    
#make_allele2pdb_table()

alleles_vs_pdb = pd.read_csv('../sandbox/alleles_vs_pdb', sep='\t', index_col=0, low_memory=False)

In [16]:
ligand_assays = pd.read_csv('../data/epitopes/mhc_ligand_full.csv', skiprows=1, low_memory=False)
ligand_assays = ligand_assays[ligand_assays['MHC allele class'] == 'I']

affinity = ligand_assays[['Description', 'Allele Name', 'Qualitative Measure', 'Quantitative measurement']]
affinity['Length'] = [len(x) for x in affinity.Description]
affinity = affinity[(affinity.Length < 12) & (affinity.Length > 7)]
affinity = affinity[affinity['Allele Name'].str.contains('HLA-[ABC]\*\S+$', regex=True)]
affinity['Allele Name'] = [x[4:] for x in affinity['Allele Name']]

In [17]:
# compare peptides with bias to terminal acids
def cmp_peptides(a, b):
        assert(len(a) == len(b))
        score = 0.0
        length = len(a)
        for i in range(length):
            if a[i] != b[i]:
                if i in [0,1,length-1]:
                    score += 1.0
                else:
                    score += 0.5
        return score

# create peptide-pdb compatibility table
def make_peptide2pdb_table():
    antigen_list = list(set(affinity['Description']))
    size = len(antigen_list)

    peptide_scores = np.zeros((size, iclass.shape[0]))
    peptide_scores.fill(np.inf)

    for counter in range(size):
        if counter % 1000 == 0:
            print(counter)
        seq = antigen_list[counter]
        length = len(seq)

        local = iclass[iclass.antigen_len == length]
        for i in local.index:
            peptide_scores[counter, i] = 0.0
            crystal_seq = local.loc[i, 'antigen_seq']
            peptide_scores[counter, i] = cmp_peptides(crystal_seq, seq)

    peptides_vs_pdb = pd.DataFrame(peptide_scores, index=antigen_list, columns=list(iclass.pdb))
    peptides_vs_pdb = peptides_vs_pdb.replace(np.inf, np.nan)
    peptides_vs_pdb.to_csv('../sandbox/peptides_vs_pdb', sep='\t')

#make_peptide2pdb_table()

peptides_vs_pdb = pd.read_csv('../sandbox/peptides_vs_pdb', sep='\t', index_col=0, low_memory=False)

In [18]:
# For each pdb in data calculate minimal rsmd of its peptide with a peptide of another pdb

def peptides_rmsd(name1, chain1, name2, chain2):
    parser = Bio.PDB.PDBParser(QUIET=True)
    super_imposer = Bio.PDB.Superimposer()
    
    str1 = parser.get_structure(id=name1, file=pdb_path+'/'+name1+'.pdb')[0]
    str1_atoms = []
    for atom in str1[chain1].get_atoms():
        if atom.get_name() == 'CA':
            str1_atoms.append(atom)

    str2 = parser.get_structure(id=name2, file=pdb_path+'/'+name2+'.pdb')[0]
    str2_atoms = []
    for atom in list(str2[chain2].get_atoms()):
        if atom.get_name() == 'CA':
            str2_atoms.append(atom)
            
    assert(len(str1_atoms) == len(str2_atoms))
    length = len(str1_atoms)
    super_imposer.set_atoms(str1_atoms, str2_atoms)
    
    return super_imposer.rms

def peptide_min_rmsd_calc():
    pdb2rmsd = {}
    errors = []

    for pdb in list(iclass.pdb):
        print('\n'+pdb+':')

        index = iclass[iclass.pdb == pdb].index[0]
        pepseq = iclass.loc[index, 'antigen_seq']
        allele = iclass.loc[index, 'a_chain_allele']
        chunk = alleles_vs_pdb.loc[alleles_vs_pdb.index.str.startswith(allele), :]
        print(pepseq)
        if pepseq not in peptides_vs_pdb.index:
            continue

        best_pdbs = (chunk.apply(min) + peptides_vs_pdb.loc[pepseq, :]/10).sort_values()

        rmsd_list = []
        for i in xrange(1, 10):
            best_pdb = best_pdbs.index[i]
            best_index = iclass[iclass.pdb == best_pdb].index[0]
            print(best_pdb)
            try:
                rmsd = peptides_rmsd(pdb, iclass.loc[index, 'antigen_id'], best_pdb, iclass.loc[best_index, 'antigen_id'])
            except AssertionError:
                print('Assertion error: incorrect peptide length')
                errors.append((pdb, best_pdb))

            rmsd_list.append(rmsd)
            #print(best_pdb+' gives '+str(rmsd))

        pdb2rmsd.update({pdb: min(rmsd_list)})

    pd.DataFrame(pdb2rmsd.values(), pdb2rmsd.keys()).to_csv('../sandbox/peptide_min_rsmd', header=None, sep='\t')

#peptide_min_rmsd_calc()

In [19]:
annotation.head()

,pdb,mhc_class,host,a_chain_id,a_chain_allele,b_chain_id,b_chain_allele,antigen_id,antigen_seq,antigen_len,antigen_name,antigen_organism,comments
0,3i6l,I,Human,D,A*24:02:85,E,B2M,F,QFKDNVILL,9,Nucleoprotein,SARS coronavirus Tor2,NaN
1,5swq,I,Human,A,A*02:01:48,B,B2M,C,CVNGSCFTV,9,neuraminidase,Influenza A virus,NaN
2,3x14,I,Human,A,B*08:02,B,B2M,C,FLRGRAYGL,9,nuclear antigen EBNA-3,Human herpesvirus 4,NaN
3,4jqx,I,Human,A,B*44:03:33,C,B2M,B,EECDSELEIKRY,12,Trans-activator protein BZLF1,Human herpesvirus 4,NaN
4,5c0d,I,Human,A,A*02:01:48,B,B2M,C,AQWGPDPAAA,10,NaN,NaN,NaN


In [ ]:
counter = 0
for item in Bio.SeqIO.parse("../data/mhc_alleles/hla/hla1class.fasta", 'fasta'):
    if counter == 427:
        print(item)
        break
    counter+=1

In [ ]:

mutate('4i48', alleledict['A*02:01'].seq, 'NLVPMVHTV', iclass)

In [ ]:
'3mrb', '4i48'

In [ ]:
iclass[iclass.pdb == '3mrb']

In [20]:
table=iclass
pdb ='4hx1'
parser = Bio.PDB.PDBParser(QUIET=True)
ppb=Bio.PDB.PPBuilder()
name = pdb; amhc = table[table.pdb == pdb].iloc[0, 3]; pept = table[table.pdb == pdb].iloc[0, 7]
struct = parser.get_structure(id=name, file=pdb_path+'/'+name+'.pdb')[0]
oldseq = str(ppb.build_peptides(struct[amhc])[0].get_sequence())
oldpep = table[table.pdb == pdb].iloc[0, 8]

In [34]:
res = ppb.build_peptides(struct[amhc])[0][0]
res.get_id()

(' ', 1, ' ')

In [ ]:
str(ppb.build_peptides(struct[amhc])[0].get_sequence())

In [ ]:
affinity.shape

In [ ]:
alleledict.keys()[427]

In [ ]:
command = ["blastp", "-query", "../testq",
                   "-subject", "../tests", "-out", 
                   "../testout", "-outfmt", str(3), "-num_alignments", str(20),
                   "-line_length", str(1000), "-ungapped", "-comp_based_stats", "F"]
if subprocess.call(command) != 0:
    print('Failed to blast')
    exit()